Julia is a Just In Time (JIT) compiled language, rather tan an interpeted one.
Julia is dynamic, built on top of LLVM. Check http://llvm.com.


Vectorization is natural in Julia, we can do away with the nester for loops.

Julia's concept of types is a key ingredient. It will compile two versions of the code, one for integer arguments and one for floating point arguments, and then insert the appropriate call in the code when it compiles the program.

Julia allows us to introspect native code.

The language is designed to make it easy to statically analyze its data types. You can usually write high-level code without fighting with the compiler about types, and still achieve superior performance.

### Personal Exercise: Plot Mandelbrot set, compare Python and Julia runtime

In [2]:
@code_warntype 3^2

Variables
  #self#::Core.Const(^)
  x::Int64
  p::Int64

Body::Int64
1 ─ %1 = Base.power_by_squaring(x, p)::Int64
└──      return %1


In [10]:
@code_warntype 3.0^(2.0)

Variables
  #self#::Core.Const(^)
  x::Float64
  y::Float64
  z::Float64

Body::Float64
1 ─       nothing
│   %2  = Base.cconvert(Base.Math.Float64, x)::Float64
│   %3  = Base.cconvert(Base.Math.Float64, y)::Float64
│   %4  = Base.unsafe_convert(Base.Math.Float64, %2)::Float64
│   %5  = Base.unsafe_convert(Base.Math.Float64, %3)::Float64
│         (z = $(Expr(:foreigncall, "llvm.pow.f64", Float64, svec(Float64, Float64), 0, :(:llvmcall), :(%4), :(%5), :(%3), :(%2))))
│   %7  = Base.Math.isnan(z)::Bool
│   %8  = (x + y)::Float64
│   %9  = Base.Math.isnan(%8)::Bool
│   %10 = !%9::Bool
│   %11 = (%7 & %10)::Bool
└──       goto #3 if not %11
2 ─       Base.Math.throw_exp_domainerror(x)
3 ┄       return z


LoadError: LoadError: MethodError: no method matching var"@time"(::LineNumberNode, ::Module)
[0mClosest candidates are:
[0m  var"@time"(::LineNumberNode, ::Module, [91m::Any[39m) at timing.jl:204
in expression starting at In[10]:2

In [4]:
f(x) = log(10.0)*x

f (generic function with 1 method)

In [5]:
@code_llvn f(4.0)

LoadError: LoadError: UndefVarError: @code_llvn not defined
in expression starting at In[5]:1

In [7]:
f(x) = x*log(x)

f (generic function with 1 method)

In [8]:
?what

search: Cwchar_t

Couldn't find what
Perhaps you meant wait, cat, hcat, stat, vcat, Char, which, where or while


No documentation found.

Binding `what` does not exist.


In [11]:
x = rand(10_000_000);

10000000-element Vector{Float64}:
 0.6522515732854011
 0.851134188970629
 0.029163888625201695
 0.7361423780928966
 0.7516624869248203
 0.7542900399902854
 0.15609290657368713
 0.19039405233214057
 0.7510607650907151
 0.2551365400978949
 0.014156844327223306
 0.7389758124422134
 0.23724527544087337
 ⋮
 0.40572037161071983
 0.7879829234378308
 0.36776479141107754
 0.6396421483201482
 0.2516765537202057
 0.9624630552249347
 0.019219770315248574
 0.6700076308927703
 0.9397380797661834
 0.9201727966080635
 0.5678188441887406
 0.9446559048803547

In [13]:
@time sum(x)

  0.039798 seconds (76.27 k allocations: 4.506 MiB, 75.72% compilation time)


5.000142740200723e6

In [14]:
@time sum(x)

  0.005367 seconds (1 allocation: 16 bytes)


5.000142740200723e6

## Avoid global variables in Julia